# Tweet Processor

**CS109a**: Fall 2018

**Authors**: Gordon Hew, Wenqin Hu, Blair Leduc

**TF**: Ken Arnold

In [1]:
import numpy as np
import pandas as pd
import matplotlib
import matplotlib.pyplot as plt
import seaborn as sns
import tweepy
import sys
import jsonpickle
import os
import zipfile
import time
from datetime import date, datetime, time
import re
import json
from pprint import pprint
import random
from dateutil import tz
from pandas.io.json import json_normalize
import requests
import gzip
import shutil
import warnings

warnings.filterwarnings('ignore')

# Parameters

In [2]:
tweets_file = os.path.join('data','last_100_timeline_tweets.json.gz')
tweets_urls_file = os.path.join('data','tweet_urls.json')

normalized_users_df_file = os.path.join('tmp','users_df.pkl')
normalized_tweets_df_file = os.path.join('tmp','tweets_df.pkl')
normalized_users_df_gz_file = os.path.join('tmp','users_df.pkl.gz')
normalized_tweets_df_gz_file = os.path.join('tmp','tweets_df.pkl.gz')

final_users_df_file = os.path.join('tmp','users_final_df.pkl')
final_tweets_df_file = os.path.join('tmp','tweets_final_df.pkl')
final_users_df_gz_file = os.path.join('tmp','users_final_df.pkl.gz')
final_tweets_df_gz_file = os.path.join('tmp','tweets_final_df.pkl.gz')

botometer_result_1000 = os.path.join('data',
                                     'botometer_result_1000random.json')

run_url_expander = False

# Extract JSON Tweets into a Pandas Data Frame

In [3]:
users_json = list()
tweets_json = list()

with gzip.open(tweets_file, 'rt') as f:
    idx = 0
    for idx, line in enumerate(f):
        line_json = json.loads(line)

        if 'user' in line_json:
            user_json = line_json.pop('user', None)
            users_json.append(user_json)

            line_json['user_id'] = user_json['id']
            line_json['id_str'] = user_json['id_str']
            line_json['screen_name'] = user_json['screen_name']

        tweets_json.append(line_json)

        if ((idx + 1) % 10000 == 0):
            print('Finished Proceessing Tweet #', idx + 1)

    print('Finished Proceessing Tweet #', idx + 1)

users_df = json_normalize(users_json)
tweets_df = json_normalize(tweets_json)

display(users_df.describe())
display(tweets_df.describe())

Finished Proceessing Tweet # 10000
Finished Proceessing Tweet # 20000
Finished Proceessing Tweet # 30000
Finished Proceessing Tweet # 40000
Finished Proceessing Tweet # 50000
Finished Proceessing Tweet # 60000
Finished Proceessing Tweet # 70000
Finished Proceessing Tweet # 80000
Finished Proceessing Tweet # 90000
Finished Proceessing Tweet # 97854


,favourites_count,followers_count,friends_count,id,listed_count,statuses_count
count,1000.000000,1000.000000,1000.000000,1.000000e+03,1000.000000,1000.000000
mean,21331.004000,1867.186000,1139.991000,3.423497e+17,25.115000,31638.967000
std,39114.780324,7344.047951,4073.343137,4.482004e+17,92.364381,59896.719907
min,0.000000,0.000000,0.000000,3.023431e+06,0.000000,1.000000
25%,1550.250000,108.750000,167.000000,4.943091e+08,0.000000,2309.750000
50%,7982.500000,363.000000,397.500000,2.814042e+09,2.000000,9648.500000
75%,23129.750000,1094.500000,908.250000,8.528221e+17,11.000000,32057.500000
max,383288.000000,100730.000000,85123.000000,1.070908e+18,1550.000000,624250.000000


,coordinates,favorite_count,geo,id,in_reply_to_status_id,in_reply_to_user_id,place,quoted_status.contributors,quoted_status.coordinates,quoted_status.favorite_count,...,retweeted_status.retweet_count,retweeted_status.user.favourites_count,retweeted_status.user.followers_count,retweeted_status.user.friends_count,retweeted_status.user.id,retweeted_status.user.listed_count,retweeted_status.user.statuses_count,retweeted_status.user.time_zone,retweeted_status.user.utc_offset,user_id
count,0.0,97854.000000,0.0,9.785400e+04,1.981000e+04,2.087000e+04,0.0,0.0,0.0,5.207000e+03,...,5.444200e+04,5.444200e+04,5.444200e+04,5.444200e+04,5.444200e+04,54442.000000,5.444200e+04,0.0,0.0,9.785400e+04
mean,NaN,1.532150,NaN,1.062750e+18,1.060179e+18,2.982529e+17,NaN,NaN,NaN,4.133277e+04,...,1.612598e+04,2.196059e+04,1.248257e+06,7.173465e+03,2.701214e+17,3002.657103,4.393735e+04,NaN,NaN,3.349681e+17
std,NaN,69.065511,NaN,4.524701e+16,5.548095e+16,4.329941e+17,NaN,NaN,NaN,1.518427e+05,...,5.590547e+04,4.538322e+04,6.206854e+06,4.133332e+04,4.176677e+17,12902.510654,2.279119e+05,NaN,NaN,4.445371e+17
min,NaN,0.000000,NaN,1.681337e+10,1.467126e+10,1.200000e+01,NaN,NaN,NaN,0.000000e+00,...,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,7.670000e+02,0.000000,0.000000e+00,NaN,NaN,3.023431e+06
25%,NaN,0.000000,NaN,1.068163e+18,1.067390e+18,2.214333e+08,NaN,NaN,NaN,1.400000e+01,...,3.500000e+01,1.140000e+03,1.692000e+03,2.130000e+02,2.286717e+08,12.000000,3.757000e+03,NaN,NaN,4.934314e+08
50%,NaN,0.000000,NaN,1.070557e+18,1.070352e+18,2.167074e+09,NaN,NaN,NaN,7.000000e+02,...,6.170000e+02,6.890000e+03,1.258100e+04,6.100000e+02,1.708861e+09,102.000000,1.413250e+04,NaN,NaN,2.787828e+09
75%,NaN,0.000000,NaN,1.071038e+18,1.070917e+18,8.150505e+17,NaN,NaN,NaN,1.228200e+04,...,7.240250e+03,2.422800e+04,1.102230e+05,1.944750e+03,7.785381e+17,747.000000,4.485200e+04,NaN,NaN,8.468366e+17
max,NaN,19464.000000,NaN,1.071283e+18,1.071281e+18,1.071065e+18,NaN,NaN,NaN,1.802781e+06,...,1.668812e+06,1.500095e+06,1.034728e+08,1.650450e+06,1.071199e+18,229738.000000,3.512209e+07,NaN,NaN,1.070908e+18


In [ ]:
# Write as binary file
users_df.to_pickle(normalized_users_df_gz_file, compression = 'gzip')
tweets_df.to_pickle(normalized_tweets_df_gz_file, compression = 'gzip')

In [ ]:
users_pkl_df = pd.read_pickle(normalized_users_df_gz_file, 
                              compression = 'gzip')
tweets_pkl_df = pd.read_pickle(normalized_tweets_df_gz_file, 
                               compression = 'gzip')

In [ ]:
display(users_pkl_df.describe())
display(users_pkl_df.head())
display(tweets_pkl_df.describe())
display(tweets_pkl_df.head())

In [ ]:
len(tweets_pkl_df.index)

# Website Classification Processing

## Extract Website URLs From Tweets

In [ ]:
tweets_pkl_df.head()

In [ ]:
tweets_with_urls_df = tweets_pkl_df[tweets_pkl_df['entities.urls']\
                                    .map(lambda urls: len(urls)) > 0]

print('tweets with links:', len(tweets_with_urls_df.index))
display(tweets_with_urls_df.head())
display(tweets_with_urls_df.describe())

In [ ]:
if run_url_expander:
    session = requests.Session()

    tweet_urls = list()

    for i,(tweet_idx, row) in enumerate(tweets_with_urls_df.iterrows()):
        # visit the URL and get the expanded site
        for url_idx, url in enumerate(row['entities.urls']):
            try:
                resp = session.head(url['expanded_url'], 
                                    allow_redirects = True)
                tweet_urls.append({'expanded_url': url['expanded_url'], 
                                   'unshortened_url': resp.url })
            except Exception as err:
                print('Unable to process url', url['expanded_url'], err)

        if ((i + 1) % 100 == 0):
            print('Finished Proceessing Tweet #', i + 1)

    print('Finished Proceessing Tweet #', i + 1)

    with open('tweet_urls.json', 'w') as f:
        for url_json in tweet_urls:
            f.write(json.dumps(url_json))
            f.write('\n')

    print('Processed', len(tweet_urls), 'URLs Successfully')

## Classify Websites Using Rules

In [ ]:
def categorize_website(url):
    
    is_twitter = url.startswith('https://twitter.com') \
        or url.startswith('https://mobile.twitter.com')
    
    if is_twitter:
        return 'Twitter'
    
    other_social_media_urls = (
        'https://i.imgur.com',
        'https://www.instagram.com',
        'https://www.facebook.com',
        'https://m.soundcloud.com',
        'https://soundcloud.com',
        'https://www.linkedin.com',
        'https://vine.co',
        'http://www.facebook.com',
    )
    
    if url.startswith(other_social_media_urls):
        return 'Other Social Media'
    
    digital_media_urls = (
        'https://www.twitch.tv',
        'https://www.youtube.com',
        'https://open.spotify.com/',
        'http://www.youtube.com',
        'http://mpg.dnset.com',
        'https://www.pscp.tv',
    )
    
    if url.startswith(digital_media_urls):
        return 'Digital Media'
    
    news_urls = (
        'https://www.nytimes.com',
        'https://www.newsweek.com',
        'https://abc7.com',
        'https://news.sky.com',
        'https://www.bbc.com',
        'https://www.sbs.com.au',
        'https://www.yahoo.com/news/',
        'https://www.yahoo.com/lifestyle/',
        'https://abcnews.go.com',
        'https://www.wsj.com',
        'https://thehill.com',
        'https://www.foxnews.com',
        'https://www.usatoday.com',
        'https://www.sun-sentinel.com',
        'https://www.washingtontimes.com',
        'https://www.washingtonpost.com',
        'http://time.com',
        'https://fox8.com',
        'https://www.huffingtonpost.com',
        'http://asiannewsservice.in',
        'http://austin.culturemap.com',
        'https://www.reuters.com',
        
        # business
        'https://www.forbes.com',
        'https://www.zerohedge.com',
        'https://thebrandboy.com',
        
        # Gaming
        'https://www.gameinformer.com',
        'https://mynintendonews.com',
        
        # Tech
        'https://www.engadget.com',
        'https://arstechnica.com',
        'https://www.zdnet.com',
        
        'https://www.travelandleisure.com',
        'http://dallas.culturemap.com',
        
        'https://www.statesman.com',
        
        'https://www.washingtonexaminer.com',
        'https://www.dailywire.com',
        'https://www.dailymail.co.uk',
        'http://www.dailymail.co.uk',
        'https://www.theblaze.com',
        'https://www.breitbart.com',
        'https://dailycaller.com',
        'https://paper.li',
        'http://www.kohimanewspaper.org',
        'http://geteducation.com.au',
        'https://morgan-magazine.com',
        'http://www.travelweekly.co.uk',
        'https://apple.news',
        'http://getstem.com.au',
        'https://www.bloomberg.com',
        'http://thefederalist.com',
        'http://getindustry.com.au',
        'https://www.thedailybeast.com',
        'https://www.hannity.com',
        'https://www.cnbc.com',
        'https://hillreporter.com',
        'https://www.politicususa.com',
        'https://www.mesopinions.com',
        'https://www.theguardian.com',
        'https://www.cpr.org',
        'https://www.cnn.com',
        'https://www.thegatewaypundit.com',
    )
    
    if url.startswith(news_urls):
        return 'News and Current Events'
    
    commercial_product = (
        'https://store.playstation.com',
        'https://www.starz.com',
        'https://www.amazon.com',
        'https://represent.com/store/pewdiepie',
        'https://itunes.apple.com',
        'https://sumall.com',
        'http://fllwrs.com/',
        'http://www.tweematic.com',
        'https://www.bible.com/', # sells bible apps
        'https://www.xbox.com',
        'https://www1.ticketmaster.com',
        'https://www.zazzle.com',
        'https://www.woodenrings.com',
        'https://www.workable.com',
        'http://www.twitlonger.com',
        
        'https://www.twittascope.com',
        'https://pages.ebay.com',
        'https://www.ebay.co.uk',
        'https://curiouscat.me',
        'https://api.curations.bazaarvoice.com',
        'https://www.exclusivetravelrates.com',
        'https://poshmark.com',
        'http://foodtronic.com',
        'https://www.etsy.com',
        'https://www.commissioncut.ca',
        'http://www.edumine.com',
        'http://smarturl.it',
        'http://naver.me',
        'https://www.gofundme.com',
        'https://www.documentcloud.org',
        'https://mailchi.mp',
        'https://fanlink.to',
        'http://nocturnalvegas.com',
        'https://www.bandsintown.com',
        'https://trakt.tv',
        'https://www.crowdfireapp.com',
        'https://gifkaro.app.link',
        'https://empire.lnk.to',
        'https://www.careerwebsite.com',
        'http://epphany.com',
    )
    
    if url.startswith(commercial_product):
        return 'Commercial Product & Services'
    
    celebrities_urls = (
        'https://people.com',
        'https://www.hollywoodreporter.com',
        'https://www.allkpop.com',
        'https://www.tmz.com',
        'https://ew.com',
        'https://www.out.com',
        'https://www.soompi.com',
        'https://www.billboard.com',
        'https://www.telltaletv.com',
        'https://entertain.naver.com',
        'https://onlyfans.com',
        'https://starcinema.abs-cbn.com',
        'https://bongino.com',
        'http://exo.smtown.com',
        'https://www.complex.com',
    )
    
    if url.startswith(celebrities_urls):
        return 'Celebrities'
    
    organizations_urls = (
        'https://www.nasa.gov',
        'https://www.justice.gov',
        'http://www.pewresearch.org',
        'https://nca2018.globalchange.gov',
        'http://cc.com/vote',
        'https://www.patriotguard.org',
        'https://www.weforum.org',
        'https://forcechange.com',
        'https://animalpetitions.org',
    )
    
    if url.startswith(organizations_urls):
        return 'Organization'    
    
    sports_urls = (
        'https://www.clevelandbrowns.com',
        'https://www.sbnation.com',
        'https://mlb.nbcsports.com',
        'https://silverandblacktoday.com',
        'https://www.mlb.com',
        'http://www.lpga.com',
        'https://www.wwe.com',
        'https://www.lpga.or.jp',
        'https://www.nhl.com',
    )
    
    if url.startswith(sports_urls):
        return 'Sports'  
    
    adult_urls = (
        'https://justfor.fans',
        'https://id10893.weebly.com',
        'https://www.manyvids.com',
    )
    
    if url.startswith(adult_urls):
        return 'Adult Content'
    
    unknown_urls = (
        'https://t1.daumcdn.net',
        'https://t.hrtye.com',
    )
    
    if url.startswith(unknown_urls):
        return 'Unknown' 
    
    return 'Uncategorized'

In [ ]:
pd.set_option('display.max_colwidth', -1)
pd.set_option('display.max_rows', 100)

urls = list()

with open(tweets_urls_file, 'r') as f:
    for url_json in f:
        url_json = json.loads(url_json)
        
        url_json['category'] \
            = categorize_website(url_json['unshortened_url'])
        
        unshortened_url = url_json['unshortened_url']
        
        if unshortened_url.startswith('https://'):
            begin_index = len('https://')
        else:
            begin_index = len('http://')
        
        try:
            index = unshortened_url.index('/', begin_index)
            url_json['domain'] = unshortened_url[:index]
        except:
            url_json['domain'] = unshortened_url
        
        urls.append(url_json)
        
urls_df = pd.DataFrame(urls)        

grouped = urls_df.groupby(['domain', 'category'])

top_100_websites_df = pd.DataFrame({'count':urls_df\
                                    .groupby(["domain", "category"])\
                                    .size()})\
                                    .nlargest(100, 'count')\
                                    .reset_index()
display(top_100_websites_df)

In [ ]:
#urls_df[['expanded_url', 'unshortened_url']].to_dict('series')
urls_dict = pd.Series(urls_df.unshortened_url.values, 
                      index = urls_df.expanded_url).to_dict()

In [ ]:
tweets_url_cat_df = tweets_pkl_df.copy(deep = True)

tweets_url_cat_df['links_to_twitter'] = 0
tweets_url_cat_df['links_to_top_social_media'] = 0
tweets_url_cat_df['links_to_top_digital_media'] = 0
tweets_url_cat_df['links_to_top_news'] = 0
tweets_url_cat_df['links_to_top_products_services'] = 0
tweets_url_cat_df['links_to_top_celebrities'] = 0
tweets_url_cat_df['links_to_top_organizations'] = 0
tweets_url_cat_df['links_to_top_sports'] = 0
tweets_url_cat_df['links_to_top_adult'] = 0

for i, (tweet_idx, row) in enumerate(tweets_url_cat_df.iterrows()):
    # visit the URL and get the expanded site
    
    if 'entities.urls' in row and len(row['entities.urls']) > 0:
    
        for url_idx, url in enumerate(row['entities.urls']):           
            
            if url['expanded_url'] in urls_dict:
            
                unshortened_url = urls_dict[url['expanded_url']]
                category = categorize_website(unshortened_url)

                if category == 'Twitter':
                    row['links_to_twitter'] = row['links_to_twitter']+1
                    tweets_url_cat_df.loc[tweet_idx, 
                                          'links_to_twitter'] \
                                            = row['links_to_twitter']
                elif category == 'Other Social Media':
                    row['links_to_top_social_media'] \
                            = row['links_to_top_social_media'] + 1
                    tweets_url_cat_df.loc[tweet_idx, 
                                'links_to_top_social_media'] \
                                    = row['links_to_top_social_media']
                elif category == 'Digital Media':
                    row['links_to_top_digital_media'] \
                        = row['links_to_top_digital_media'] + 1
                    tweets_url_cat_df.loc[tweet_idx, 
                                'links_to_top_digital_media'] \
                                    = row['links_to_top_digital_media']              
                elif category == 'News and Current Events':
                    row['links_to_top_news']=row['links_to_top_news']+1
                    tweets_url_cat_df.loc[tweet_idx, 
                                          'links_to_top_news'] \
                                            = row['links_to_top_news']              
                elif category == 'Commercial Product & Services':
                    row['links_to_top_products_services'] \
                        = row['links_to_top_products_services'] + 1
                    tweets_url_cat_df.loc[tweet_idx, 
                            'links_to_top_products_services'] \
                                = row['links_to_top_products_services']           
                elif category == 'Celebrities':
                    row['links_to_top_celebrities'] \
                        = row['links_to_top_celebrities'] + 1
                    tweets_url_cat_df.loc[tweet_idx, 
                                'links_to_top_celebrities'] \
                                    = row['links_to_top_celebrities']
                elif category == 'Organization':
                    row['links_to_top_organizations'] \
                        = row['links_to_top_organizations'] + 1
                    tweets_url_cat_df.loc[tweet_idx, 
                                'links_to_top_organizations'] \
                                    = row['links_to_top_organizations']          
                elif category == 'Sports':
                    row['links_to_top_sports'] \
                        = row['links_to_top_sports'] + 1
                    tweets_url_cat_df.loc[tweet_idx, 
                                        'links_to_top_sports'] \
                                            = row['links_to_top_sports']            
                elif category == 'Adult Content':
                    row['links_to_top_adult'] \
                        = row['links_to_top_adult'] + 1
                    tweets_url_cat_df.loc[tweet_idx, 
                                    'links_to_top_adult'] \
                                            = row['links_to_top_adult']

## Append Bot-o-meter Score to User Accounts

In [ ]:
with open(botometer_result_1000, 'rt') as bot_read:
    botometer_result = json.load(bot_read)
    
botometer_results_df = json_normalize(botometer_result)
users_final_df = users_pkl_df.copy(deep = True)
botometer_results_df = botometer_results_df[['user.id_str', 
                                             'scores.universal']]
users_final_df = users_final_df.merge(botometer_results_df, 
                                      left_on='id_str', 
                                      right_on='user.id_str', 
                                      how = 'left')

## Append Bot-o-meter Score to Tweets

In [ ]:
tweets_final_df = tweets_url_cat_df.copy(deep = True)
botometer_results_df = botometer_results_df[['user.id_str', 
                                             'scores.universal']]
tweets_final_df = tweets_final_df.merge(botometer_results_df, 
                                        left_on='id_str', 
                                        right_on='user.id_str', 
                                        how = 'left')

# Write out Final Data Frames for Aggregation Processing

In [ ]:
users_final_df.to_pickle(final_users_df_gz_file, compression = 'gzip')
tweets_final_df.to_pickle(final_tweets_df_gz_file, compression = 'gzip')